In [ ]:
# Import system packages
import ipywidgets as widgets
import pandas as pd
import math
from matplotlib import pyplot as plt
import arcpy
%pprint
# Define dataframe
df = pd.read_excel(r"C:\XXXXX")
# Define sliders and their handler function
sliderfs = widgets.FloatSlider(value=0,min=0,max=70,step=0.1)
slidermw = widgets.FloatSlider(value=0,min=0,max=70,step=0.01)
def handler(change):
    slider.value = change['new']

In [ ]:
# Get user flood inputs
print("Compare a flood to your dataset")
location = input("Location: ")
print("Flood stage (ft): ")
display(sliderfs)
print("Max water level (ft): ")
display(slidermw)

In [ ]:
# Calculate heights above flood stage for user inputs and for the dataframe
# Get values of user inputs and make lists for columns in dataframe
sliderfs.observe(handler, 'value')
slidermw.observe(handler, 'value')
fstage = float(sliderfs.value)
maxwater = float(slidermw.value)
hafs = maxwater - fstage
locationcol = df["Location"]
locationlist = locationcol.tolist()
locationlist.append(location)
maxwatercol = df["Max Water Level (ft)"]
fstagecol = df["Flood Stage (ft)"]
maxwaterlist = maxwatercol.tolist()
fstagelist = fstagecol.tolist()
# Calculate list of HAFS values for bar chart heights
hafslist = []
ziphafsl = zip(maxwaterlist, fstagelist)
for maxwaterlist_i, fstagelist_i in ziphafsl:
    hafslist.append(maxwaterlist_i-fstagelist_i)
hafslist = [round(h, 2) for h in hafslist]
hafslist.append(hafs)
# Calculate separate list of HAFS values only for dataframe
hafscol = []
ziphafsc = zip(maxwaterlist, fstagelist)
for maxwaterlist_i, fstagelist_i in ziphafsc:
    hafscol.append(maxwaterlist_i-fstagelist_i)
hafscol = [round(h, 2) for h in hafscol]

In [ ]:
# Ask user if they want to compare HAFS
choice = input("Graphically compare " + location + "'s Height Above Flood Stage with the dataframe? Y/N ")
if choice == "Y":
    # Graph to compare height above flood stage from inputs to heights above flood stage from dataframe
    print(location + "'s Height Above Flood Stage compared with the dataframe:")
    bargraph = plt.barh(locationlist, hafslist)
else:
    print("Next, discover the dataframe's HAFS Index values.")

In [ ]:
# Calculate index values from new HAFS column of dataframe
df["Height Above Flood Stage (ft)"] = hafscol
hafscol = df["Height Above Flood Stage (ft)"]
meanHAFS = hafscol.mean()
countHAFS = hafscol.count()
HAFSlist = hafscol.tolist()
sqtotal = 0
for h in HAFSlist:
    sq = (h - meanHAFS)**2
    sqtotal += sq
stdevHAFS = math.sqrt((sqtotal)/(countHAFS - 1))
HAFSindex = []
for h in HAFSlist:
    zscoreHAFS = (h - meanHAFS)/stdevHAFS 
    indexHAFS = round((50+(20*zscoreHAFS))*2, 2)
    HAFSindex.append(indexHAFS)
df["HAFS Index"] = HAFSindex
display(df)

In [ ]:
# Calculate comparative index value from HAFS value of user inputs
def indexHAFS(HAFS):
    zscore = (HAFS - meanHAFS)/stdevHAFS
    index = (50+(20*zscore))*2
    return index
print("Compared to this flood event, your", location, "flood would have an HAFS Index of", round(indexHAFS(hafs), 2))

In [ ]:
# Make lists from LAT and LONG columns and empty lists for converted values
LATlist = df["LAT"].tolist()
LONGlist = df["LONG"].tolist()
newLATlist = []
newLONGlist = []
# Check if N/S latitudes and E/W longitudes need to be converted to +/- decimal degree values
try:    
    for l in LATlist:
        if "N" in l:
            newl = float(l.replace(' N', ''))
            newLATlist.append(newl)
        elif "S" in l:
            newl = -(float(l.replace(' S', '')))
            newLATlist.append(newl)
        else:
            newLATlist.append(float(l))
    for l in LONGlist:
        if "E" in l:
            newl = float(l.replace(' E', ''))
            newLONGlist.append(newl)
        elif "W" in l:
            newl = -(float(l.replace(' W', '')))
            newLONGlist.append(newl)
        else:
            newLONGlist.append(float(l))
    df["LAT"] = newLATlist
    df["LONG"] = newLONGlist
# Otherwise keep them the same
except TypeError:
    df["LAT"] = LATlist
    df["LONG"] = LONGlist

In [ ]:
# Create rivers selected by state layer
arcpy.env.workspace = r"C:\XXXXX"
USrivers = "USA_Rivers_and_Streams.shp"
where_clause = "State = 'LA'"
selectedrivers = arcpy.SelectLayerByAttribute_management(USrivers, "NEW_SELECTION", where_clause)
arcpy.CopyFeatures_management(selectedrivers, "LArivers")
# Create gauge points layer
df.to_csv(r"C:\XXXXX", index = False)
in_table = r"C:\XXXXX"
out_fc = "Water_Height_Gauges"
x_coords = "LONG"
y_coords = "LAT"
arcpy.management.XYTableToPoint(in_table, out_fc, x_coords, y_coords)
# Create LA rivers selected by intersection with gauge points layer
floodrivers = arcpy.SelectLayerByLocation_management("LArivers", "INTERSECT", "Water_Height_Gauges", "2 Miles", "NEW_SELECTION", 
                                                     "NOT_INVERT")
arcpy.CopyFeatures_management(floodrivers, "Flood_Rivers")

In [ ]:
aprx = arcpy.mp.ArcGISProject(r"C:\XXXXX")
map = aprx.listMaps("Map")[0]
# Add Flood_Rivers layer to map
map.addDataFromPath(r"C:\XXXXX")
# Add Water_Height_Gauges layer to map
map.addDataFromPath(r"C:\XXXXX")

In [ ]:
# Change Flood_Rivers layer symbology color to blue
layer = map.listLayers("Flood_Rivers")[0]
sym = layer.symbology
sym.renderer.symbol.color = {'RGB':  [0, 0, 255, 100]}
layer.symbology = sym
# Change Water_Height_Gauges layer symbology color to red
layer2 = map.listLayers("Water_Height_Gauges")[0]
sym2 = layer2.symbology
sym2.renderer.symbol.color = {'RGB': [255, 0, 0, 100]}
layer2.symbology = sym2

# Save project
aprx.save()
del aprx